In [13]:
from src.preprocessor.text_preprocessor import TextPreprocessing
import re
import string
import pandas as pd
from tqdm import tqdm
from typing import Union
from pyvi import ViTokenizer
from underthesea import ner
from src.preprocessor.vocab.stopwords import STOP_WORDS
from src.preprocessor.vocab.legal_dict import LEGAL_DICT
from src.preprocessor.vocab.duties_dict import DUTIES
from src.preprocessor.vocab.special_terms import SPECIAL_TERMS
from src.preprocessor.legal_processing.legal_terms_tokenize import terms_of_law
from src.preprocessor.legal_processing.duties_tokenize import duties_terms, ner_tokenize
from IPython.display import display, Markdown

In [105]:
txt = """
Ban hành tiêu chí, quy trình thủ tục công nhận xã An toàn khu, vùng An toàn khu như sau:\n...\n2. Tiêu chí xác định xã An toàn khu, vùng An toàn khu\n...\nb) Vùng An toàn khu phải có đủ 02 tiêu chí sau:\n- Tiêu chí 1: Vùng có địa bàn thuộc 01 hoặc một số đơn vị hành chính cấp huyện liền kề của 01 hoặc một số tỉnh, thành phố trực thuộc trung ương; các đơn vị hành chính cấp xã trong vùng này có cơ sở và phong trào cách mạng vững mạnh trong kháng chiến chống Pháp, chống Mỹ.\n- Tiêu chí 2: Có từ 30% trở lên số đơn vị hành chính cấp xã trong vùng được công nhận xã An toàn khu, trong đó có đơn vị hành chính cấp xã được phong tặng Danh hiệu Anh hùng lực lượng vũ trang hoặc có các Di tích lịch sử cách mạng cấp quốc gia, các công trình di tích lịch sử cách mạng được các tổ chức, cơ quan từ cấp Khu ủy, Quân khu trở lên đầu tư xây dựng trên địa bàn.
2. Nhiệm vụ\n2.1. Nghiên cứu, đề xuất và thực hiện\na) Nghiên cứu, đề xuất và giúp ban chấp hành, ban thường vụ, thường trực tỉnh ủy tổ chức thực hiện chương trình công tác; xây dựng, tổ chức thực hiện quy chế làm việc; sắp xếp chương trình công tác của đồng chí bí thư, các phó bí thư và một số hoạt động của các đồng chí ủy viên ban thường vụ tỉnh ủy theo quy chế làm việc và chương trình công tác; thực hiện công tác đối ngoại của tỉnh ủy; phối hợp và điều hòa hoạt động của các cơ quan tham mưu, giúp việc tỉnh ủy. Chủ trì, phối hợp tham mưu và tổ chức thực hiện công tác cải cách hành chính trong Đảng.\nb) Tổng hợp tình hình tổ chức thực hiện nghị quyết, quyết định, chỉ thị, quy định, quy chế, kết luận của Trung ương, của tỉnh ủy và hoạt động của các cấp ủy, đảng đoàn, ban cán sự đảng, các cơ quan thuộc tỉnh ủy. Thông báo ý kiến kết luận, chỉ đạo của ban thường vụ, thường trực tỉnh ủy, bí thư, phó bí thư và theo dõi, đôn đốc việc thực hiện. Theo dõi, đôn đốc các tổ chức đảng trực thuộc tỉnh ủy, các cơ quan liên quan thực hiện chế độ thông tin báo cáo theo quy định. Thực hiện công tác thông tin phục vụ sự lãnh đạo của tỉnh ủy; chế độ cung cấp thông tin cho cấp ủy viên và các tổ chức. Giúp tỉnh ủy thực hiện chế độ thông tin, báo cáo định kỳ và đột xuất.\nc) Tiếp nhận, phát hành và quản lý các tài liệu, văn bản đến và đi của tỉnh ủy và các cơ quan tham mưu, giúp việc tỉnh ủy; quản lý, khai thác mạng cơ yếu của tỉnh ủy. Tổ chức quản lý, khai thác, ứng dụng công nghệ thông tin, bảo vệ mạng thông tin diện rộng của đảng bộ.\nd) Quản lý, tổ chức khai thác tài liệu Phông Lưu trữ Đảng Cộng sản Việt Nam thuộc thẩm quyền thu thập của lưu trữ lịch sử đảng bộ tỉnh, bao gồm tài liệu lưu trữ của các cơ quan, tổ chức đảng, tổ chức chính trị - xã hội.\nđ) Trực tiếp quản lý tài chính, tài sản, chi tiêu ngân sách đảng; hành chính, quản trị, tài vụ phục vụ hoạt động của cấp ủy và các cơ quan tham mưu, giúp việc tỉnh ủy. Thực hiện thẩm tra quyết toán tài chính ngân sách đảng của các tổ chức đảng và đảng bộ trực thuộc. Bảo đảm điều kiện vật chất, phương tiện, trang thiết bị cho hoạt động của tỉnh ủy và các cơ quan tham mưu, giúp việc tỉnh ủy; đồng thời, bảo đảm tài chính và cơ sở vật chất khác cho các cơ quan trực thuộc tỉnh ủy theo phân công, phân cấp.\ne) Tham gia tổ chức, phục vụ đại hội đảng bộ tỉnh; chủ trì phục vụ hội nghị tỉnh ủy, ban thường vụ tỉnh ủy và các hội nghị do thường trực tỉnh ủy triệu tập, các cuộc làm việc của bí thư, phó bí thư tỉnh ủy.\ng) Sơ kết, tổng kết công tác văn phòng cấp ủy.\n2.2. Hướng dẫn, kiểm tra, giám sát\na) Hướng dẫn, bồi dưỡng nghiệp vụ công tác văn phòng, công tác tài chính, quản lý tài sản của Đảng cho các tổ chức đảng trực thuộc tỉnh ủy và văn phòng cấp ủy cấp dưới; nghiệp vụ công tác văn thư, lưu trữ của các cơ quan, tổ chức đảng, đoàn thể chính trị - xã hội ở địa phương theo quy định của pháp luật về lưu trữ, các quy định của Ban Bí thư và hướng dẫn của Văn phòng Trung ương Đảng; công nghệ thông tin, nghiệp vụ công tác cơ yếu cho văn phòng cấp ủy cấp dưới và các cơ quan đảng thuộc tỉnh ủy.\nb) Thực hiện kiểm tra, giám sát về nghiệp vụ văn phòng và công tác tài chính - kế toán ở các đảng bộ và tổ chức trực thuộc tỉnh ủy.\nc) Chủ trì, phối hợp với Ủy ban kiểm tra tỉnh ủy, ban tổ chức tỉnh ủy và các cơ quan liên quan giúp tỉnh ủy, ban thường vụ tỉnh ủy theo dõi, kiểm tra, giám sát việc thực hiện quy chế làm việc của tỉnh ủy, ban thường vụ tỉnh ủy.\n2.3. Thẩm định, thẩm tra\na) Đề án, văn bản của các cơ quan, tổ chức trước khi trình tỉnh ủy, ban thường vụ, thường trực tỉnh ủy về: Yêu cầu, phạm vi, quy trình, thẩm quyền ban hành và thể thức văn bản.\nb) Nội dung đề án, văn bản thuộc lĩnh vực kinh tế - xã hội, quốc phòng - an ninh, đối ngoại và các lĩnh vực khác theo sự phân công của thường trực tỉnh ủy trước khi trình tỉnh ủy, ban thường vụ tỉnh ủy, thường trực tỉnh ủy.\n2.4. Phối hợp\na) Với các cơ quan có liên quan nghiên cứu, đề xuất một số chủ trương của tỉnh ủy, ban thường vụ tỉnh ủy về cơ chế, chính sách kinh tế - xã hội, quốc phòng - an ninh, đối ngoại; tham gia ý kiến với cơ quan nhà nước trong việc cụ thể hóa chủ trương, chính sách phát triển kinh tế - xã hội ở địa phương.\nb) Với các cơ quan liên quan xây dựng một số đề án, văn bản, chương trình hành động do ban thường vụ tỉnh ủy, thường trực tỉnh ủy giao; biên tập hoặc thẩm định văn bản trước khi ban thường vụ tỉnh ủy, thường trực tỉnh ủy ban hành.\nc) Với ban tổ chức tỉnh ủy trong công tác cán bộ theo quy định của ban thường vụ tỉnh ủy về phân cấp quản lý cán bộ và giới thiệu cán bộ ứng xử; trong việc quản lý tổ chức bộ máy, biên chế và thực hiện chế độ, chính sách đối với cán bộ, công chức, viên chức, người lao động của văn phòng tỉnh ủy theo phân cấp.\nd) Với các cơ quan tham mưu, giúp việc của tỉnh ủy, đảng đoàn, ban cán sự đảng, đảng ủy trực thuộc tỉnh ủy để tham mưu giúp tỉnh ủy, ban thường vụ tỉnh ủy, thường trực tỉnh ủy chỉ đạo, kiểm tra, giám sát, sơ kết, tổng kết việc thực hiện nghị quyết, chỉ thị, quy định, quy chế, quyết định, kết luận của Trung ương và của tỉnh ủy về công tác xây dựng đảng; về kinh tế - xã hội, quốc phòng - an ninh, đối ngoại; việc thực hiện chủ trương, chế độ, nguyên tắc quản lý tài chính, tài sản của tỉnh ủy.\nđ) Với cơ quan chức năng nhà nước trong việc quản lý quy hoạch, kế hoạch đầu tư xây dựng cơ bản và thực hiện dự án cho các cơ quan đảng theo quy định của pháp luật.\ne) Với ban nội chính tỉnh ủy giúp thường trực tỉnh ủy tổ chức tiếp công dân.\n2.5. Thực hiện một số nhiệm vụ khác do ban thường vụ tỉnh ủy, thường trực tỉnh ủy giao.
Yêu cầu về dung sai\nCho phép dung sai về chất lượng và kích cỡ quả trong mỗi bao gói sản phẩm (hoặc mỗi lô hàng sản phẩm ở dạng rời) không đáp ứng các yêu cầu quy định của mỗi hạng.\n4.1 Dung sai về chất lượng\n...\n4.1.3 Hạng II\nCho phép 10 % số lượng hoặc khối lượng mướp quả tươi không đáp ứng các yêu cầu của hạng II cũng như các yêu cầu tối thiểu, trừ sản phẩm bị thối hoặc hư hỏng bất kỳ khác dẫn đến không thích hợp cho việc sử dụng.\n...
Thi hài, hài cốt, tro cốt được phép đưa về Việt Nam\n1.
"""


In [34]:
from src.preprocessor.vocab.numeral_currency import CURRENCY



1 000.000 là một triệu và 10.000.000 là chục triệu.


In [44]:
import re
import string




Đây là 1 000.000 và Còn có 000.000 là triệu.


In [122]:
def url_remover(paragraph: str) -> str:
    """Remove URLs within parentheses from text."""
    pattern = r'\([^)]*http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+[^)]*\)'
    
    def replace_url(match):
        content = match.group(0)
        cleaned = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', content)
        return cleaned if cleaned.strip('() ') else ''
        
    return re.sub(pattern, replace_url, paragraph)

def punctuation_remover(paragraph: str) -> str:
    """Remove punctuation marks from text and handle currency conversions."""
    # Loại bỏ dấu ngoặc khỏi số
    paragraph = re.sub(r"\((\d+)\)", r"\1", paragraph)
    paragraph = re.sub(r"\w+\)", " ", paragraph)

    # Tách các từ trong đoạn văn
    words = paragraph.split()
    updated_words = []

    # Xử lý các từ liên quan đến số và tiền tệ
    for item in words:
        # Nếu từ kết thúc bằng ')' hoặc '.' và bắt đầu bằng số từ 1-9, loại bỏ từ đó
        if item.endswith((')', '.')) and item[0] in '123456789':
            continue

        # Thay thế chính xác từ khớp với từ điển CURRENCY
        if item in CURRENCY:
            updated_words.append(CURRENCY[item])
            continue

        # Tìm và thay thế các trường hợp chứa mẫu trong từ điển CURRENCY
        for key, value in CURRENCY.items():
            if key in item:
                item = item.replace(key, f" {value}")
                break

        updated_words.append(item)

    # Kết hợp lại đoạn văn đã được cập nhật
    paragraph = ' '.join(updated_words)

    # Loại bỏ các dấu câu (trừ '/' và '.')
    for punc in string.punctuation:
        if punc == ":":
            paragraph = paragraph.replace(punc, ".")
        elif punc == '-':
            paragraph = paragraph.replace(punc, "")
        elif punc not in ["/", "."]:
            paragraph = paragraph.replace(punc, " ")

    # Loại bỏ khoảng trắng thừa
    return re.sub(r"\s+", " ", paragraph).strip()

def line_breaker_remover(paragraph: str) -> str:
        """Remove line breaks from text."""
        para = re.sub(r"\\n+", ". ", paragraph)
        para = re.sub(r"\n+", ". ", paragraph)
        para = re.sub(r"\.\.\.", ". ", para)
        para = re.sub(r'\.{1,}', '.', para)
        return para.replace("  ", " ")

def lowercase_standardizer(paragraph: str) -> str:
    """Convert text to lowercase."""
    return paragraph.lower()

def white_space_remover(paragraph: str) -> str:
        """Remove extra whitespace from text."""
        para = paragraph.replace("  ", " ")
        return re.sub(r"\s{2,}", " ", para).strip()
def legal_text_tokenizer(paragraph: str) -> str:
        """Tokenize legal terms in text."""
        for phrase, replacement in LEGAL_DICT.items():
            paragraph = paragraph.replace(phrase, replacement)
        
        paragraph = terms_of_law(paragraph)
        paragraph = duties_terms(paragraph)
        paragraph = dupplicated_char_remover(paragraph)
        return paragraph

def text_tokenizer(paragraph: str) -> str:
        """Tokenize regular text."""
        paragraph = ner_tokenize(paragraph)
        for phrase, replacement in SPECIAL_TERMS.items():
            paragraph = paragraph.replace(phrase, replacement)
        paragraph = preprocess_pyvi(paragraph)
        paragraph = ViTokenizer.tokenize(paragraph)
        paragraph = postprocess_pyvi(paragraph)
        print("12-----------")
        
        return paragraph
def stopword_remover(paragraph: str) -> str:
        """Remove stopwords from text."""
        return " ".join([word for word in paragraph.split() if word not in STOP_WORDS]).strip()

    
def remove_isolated_numbers(text):
    # Tách văn bản thành các từ
    words = text.split()
    filtered_words = []

    for word in words:
        try:
            # Nếu từ có thể chuyển thành số nguyên, bỏ qua từ đó
            if int(word.replace(',', '').replace('.', '')):
                continue
        except ValueError:
            # Nếu từ không thể chuyển thành số nguyên, kiểm tra thêm các điều kiện khác
            # Pattern để nhận diện số có dấu / ở trước hoặc sau
            has_slash_number = re.findall(r'(?<=/)\d+|\d+(?=/)', word)

            # Pattern để nhận diện từ có chứa ký tự khác
            contains_mix = re.search(r'[^\d.,\s]', word)
            
            # Giữ lại từ nếu có ký tự khác hoặc chứa số có dấu /
            if contains_mix or has_slash_number:
                filtered_words.append(word)

    # Kết hợp lại thành văn bản
    return ' '.join(filtered_words)
# stopword_remover(text_tokenizer(legal_text_tokenizer(white_space_remover(lowercase_standardizer(line_breaker_remover(punctuation_remover(
display(Markdown(remove_isolated_numbers(text_tokenizer(legal_text_tokenizer(white_space_remover(lowercase_standardizer(line_breaker_remover(punctuation_remover(punctuation_remover(url_remover(txt)))))))))))

12-----------


ban_hành tiêu_chí quy_trình thủ_tục công_nhận xã an_toàn_khu vùng an_toàn khu như sau tiêu_chí xác_định xã an_toàn_khu vùng an_toàn_khu vùng an_toàn khu phải có đủ tiêu_chí sau tiêu_chí vùng có địa_bàn thuộc hoặc một_số đơn_vị hành_chính cấp huyện liền kề của hoặc một_số tỉnh_thành_phố trực_thuộc trung_ương các đơn_vị hành_chính cấp xã trong vùng này có cơ_sở và phong_trào cách_mạng vững_mạnh trong kháng_chiến chống pháp chống mỹ tiêu_chí có từ trở_lên số đơn_vị hành_chính cấp xã trong vùng được công_nhận xã an_toàn_khu trong đó có đơn_vị hành_chính cấp xã được phong_tặng danh_hiệu anh hùng lực_lượng vũ_trang hoặc có các di_tích lịch_sử cách_mạng cấp quốc_gia các công_trình di_tích lịch_sử cách_mạng được các tổ_chức cơ_quan từ cấp khu_ủy quân_khu trở_lên đầu_tư xây_dựng trên địa_bàn nhiệm_vụ nghiên_cứu đề_xuất và thực_hiện nghiên_cứu đề_xuất và giúp ban chấp_hành ban thường_vụ thường_trực tỉnh ủy tổ_chức thực_hiện chương_trình công_tác xây_dựng tổ_chức thực_hiện quy_chế làm_việc sắp_xếp chương_trình công_tác của đồng_chí bí_thư các phó bí_thư và một_số hoạt_động của các đồng_chí_ủy viên ban thường_vụ tỉnh_ủy theo quy_chế làm_việc và chương_trình công_tác thực_hiện công_tác đối_ngoại của tỉnh ủy phối_hợp và điều hòa hoạt_động của các cơ_quan tham_mưu giúp_việc tỉnh_ủy chủ_trì phối_hợp tham_mưu và tổ_chức thực_hiện công_tác cải_cách hành_chính trong đảng tổng_hợp tình_hình tổ_chức thực_hiện nghị_quyết quyết_định chỉ_thị quy_định quy_chế kết_luận của trung_ương của tỉnh_ủy và hoạt_động của các cấp ủy đảng_đoàn ban cán_sự đảng các cơ_quan thuộc tỉnh_ủy thông_báo ý_kiến kết_luận chỉ_đạo của ban thường_vụ thường_trực tỉnh ủy bí_thư phó bí_thư và theo_dõi đôn_đốc việc thực_hiện theo_dõi đôn_đốc các tổ_chức đảng trực_thuộc tỉnh_ủy các cơ_quan liên_quan thực_hiện chế_độ thông_tin báo_cáo theo quy_định thực_hiện công_tác thông_tin phục_vụ sự lãnh_đạo của tỉnh ủy chế_độ cung_cấp thông_tin cho cấp ủy_viên và các tổ_chức giúp tỉnh_ủy thực_hiện chế_độ thông_tin báo_cáo định_kỳ và đột_xuất tiếp_nhận phát_hành và quản_lý các tài_liệu văn_bản đến và đi của tỉnh_ủy và các cơ_quan tham_mưu giúp_việc tỉnh ủy quản_lý khai_thác mạng cơ_yếu của tỉnh_ủy tổ_chức quản_lý khai_thác ứng_dụng công_nghệ thông_tin bảo_vệ mạng thông_tin diện rộng của đảng_bộ quản_lý tổ_chức khai_thác tài_liệu phông lưu_trữ đảng cộng_sản việt_nam thuộc thẩm_quyền thu_thập của lưu_trữ lịch_sử đảng_bộ tỉnh bao_gồm tài_liệu lưu_trữ của các cơ_quan tổ_chức đảng tổ_chức chính_trị xã_hội trực_tiếp_quản_lý_tài_chính tài_sản chi_tiêu ngân_sách đảng hành_chính quản_trị tài_vụ phục_vụ hoạt_động của cấp ủy và các cơ_quan tham_mưu giúp_việc tỉnh_ủy thực_hiện thẩm_tra quyết_toán tài_chính ngân_sách đảng của các tổ_chức đảng và đảng_bộ trực_thuộc bảo_đảm điều_kiện vật_chất phương_tiện trang thiết_bị cho hoạt_động của tỉnh_ủy và các cơ_quan tham_mưu giúp_việc tỉnh ủy đồng_thời bảo_đảm tài_chính và cơ_sở vật_chất khác cho các cơ_quan trực_thuộc tỉnh_ủy theo phân_công phân_cấp tham_gia tổ_chức phục_vụ đại_hội đảng_bộ tỉnh chủ_trì phục_vụ hội_nghị tỉnh ủy_ban thường_vụ tỉnh_ủy và các hội_nghị do thường_trực tỉnh_ủy triệu_tập các cuộc làm_việc của bí_thư phó bí_thư tỉnh_ủy sơ_kết tổng_kết công_tác văn_phòng cấp ủy hướng_dẫn kiểm_tra giám_sát hướng_dẫn bồi_dưỡng nghiệp_vụ công_tác văn_phòng công_tác tài_chính quản_lý tài_sản của đảng cho các tổ_chức đảng trực_thuộc tỉnh_ủy và văn_phòng cấp ủy cấp dưới nghiệp_vụ công_tác văn_thư_lưu_trữ của các cơ_quan tổ_chức đảng_đoàn_thể chính_trị xã_hội ở địa_phương theo quy_định của pháp_luật về lưu_trữ các quy_định của ban bí_thư và hướng_dẫn của Văn_phòng_Trung_ương_Đảng công_nghệ thông_tin nghiệp_vụ công_tác cơ_yếu cho văn_phòng cấp ủy cấp dưới và các cơ_quan đảng thuộc tỉnh_ủy thực_hiện kiểm_tra giám_sát về nghiệp_vụ văn_phòng và công_tác tài_chính kế_toán ở các đảng_bộ và tổ_chức trực_thuộc tỉnh_ủy chủ_trì phối_hợp với ủy_ban kiểm_tra tỉnh ủy_ban tổ_chức tỉnh_ủy và các cơ_quan liên_quan giúp tỉnh ủy_ban thường_vụ tỉnh_ủy theo_dõi kiểm_tra giám_sát việc thực_hiện quy_chế làm_việc của tỉnh ủy_ban thường_vụ tỉnh_ủy thẩm_định thẩm_tra đề_án văn_bản của các cơ_quan tổ_chức trước khi trình tỉnh ủy_ban thường_vụ thường_trực tỉnh_ủy về yêu_cầu phạm_vi quy_trình thẩm_quyền ban_hành và thể_thức văn_bản nội_dung đề_án văn_bản thuộc lĩnh_vực kinh_tế xã_hội quốc_phòng an_ninh đối_ngoại và các lĩnh_vực khác theo sự phân_công của thường_trực tỉnh_ủy trước khi trình tỉnh ủy_ban thường_vụ tỉnh_ủy thường_trực tỉnh_ủy phối_hợp với các cơ_quan có liên_quan nghiên_cứu đề_xuất một_số chủ_trương của tỉnh ủy_ban thường_vụ tỉnh_ủy về cơ_chế chính_sách kinh_tế xã_hội quốc_phòng an_ninh đối_ngoại tham_gia ý_kiến với cơ_quan nhà_nước trong việc cụ_thể_hóa chủ_trương chính_sách phát_triển kinh_tế xã_hội ở địa_phương với các cơ_quan liên_quan xây_dựng một_số đề_án văn_bản chương_trình hành_động do ban thường_vụ tỉnh_ủy thường_trực tỉnh ủy giao biên_tập hoặc thẩm_định văn_bản trước khi ban thường_vụ tỉnh_ủy thường_trực tỉnh ủy ban_hành với ban tổ_chức tỉnh_ủy trong công_tác cán_bộ theo quy_định của ban thường_vụ tỉnh_ủy về phân_cấp quản_lý cán_bộ và giới_thiệu cán_bộ ứng_xử trong việc quản_lý tổ_chức bộ_máy biên_chế và thực_hiện chế_độ chính_sách đối_với cán_bộ công_chức viên_chức người lao_động của văn_phòng tỉnh_ủy theo phân_cấp với các cơ_quan tham_mưu giúp_việc của tỉnh ủy đảng_đoàn ban cán_sự đảng đảng_ủy trực_thuộc tỉnh_ủy để tham_mưu giúp tỉnh ủy_ban thường_vụ tỉnh_ủy thường_trực tỉnh_ủy chỉ_đạo kiểm_tra giám_sát sơ_kết tổng_kết việc thực_hiện nghị_quyết chỉ_thị quy_định quy_chế quyết_định kết_luận của trung_ương và của tỉnh_ủy về công_tác xây_dựng đảng về kinh_tế xã_hội quốc_phòng an_ninh đối_ngoại việc thực_hiện chủ_trương chế_độ nguyên_tắc quản_lý_tài_chính tài_sản của tỉnh_ủy với cơ_quan_chức_năng nhà_nước trong việc quản_lý quy_hoạch kế_hoạch đầu_tư xây_dựng cơ_bản và thực_hiện dự_án cho các cơ_quan đảng theo quy_định của pháp_luật với ban nội_chính tỉnh ủy giúp thường_trực tỉnh ủy tổ_chức tiếp công_dân thực_hiện một_số nhiệm_vụ khác do ban thường_vụ tỉnh_ủy thường_trực tỉnh ủy giao yêu_cầu về dung_sai cho phép dung_sai về chất_lượng và kích_cỡ quả trong mỗi bao_gói sản_phẩm hoặc mỗi lô hàng sản_phẩm ở dạng không đáp_ứng các yêu_cầu quy_định của mỗi hạng dung_sai về chất_lượng hạng ii cho phép số_lượng hoặc khối_lượng mướp quả tươi không đáp_ứng các yêu_cầu của hạng ii cũng như các yêu_cầu tối_thiểu trừ sản_phẩm bị thối hoặc hư_hỏng bất_kỳ khác dẫn đến không thích_hợp cho việc sử_dụng thi_hài hài_cốt tro cốt được phép đưa về việt_nam

In [97]:
import re

def preprocess_pyvi(text):
    # Bao bọc các cụm từ có dạng ký_tự/ký_tự/... bằng {}
    text = re.sub(r'(\S+/\S+(/\S+)*)', r'{\1}', text)
    return text

def postprocess_pyvi(text):
    # Khôi phục lại các cụm từ được bao bọc bởi {}
    text = re.sub(r'\{\s*(\S+(?:\s*/\s*\S+)*)\s*\}', lambda m: m.group(1).replace(' ', ''), text)
    return text

